<h1 align="center">
Preparo dos textos dos resumos das teses e dissertações <br/>
<img src="https://dadosabertos.capes.gov.br/img/caixa.png"  alt="Dados Capes"/>
</h1>



Em nosso conjunto de dados possuímos informações sobre teses e dissertações defendidas no período de 1987-2022. No notebook de [download de metadados](notebooks/1.download_catalogos.ipynb), realizamos o download e junção de todos os conjuntos.

Desejamos extrair informações e estatísticas sobre o conteúdo das teses e dissertações, para isso utilizamos a coluna `DS_RESUMO`. Antes de realizar a análise, é necessário realizar um pré-processamento dos textos, removendo elementos que não são relevantes para a análise. Nesse notebook, exploramos os textos dos resumos em busca de padrões e elementos que podem ser removidos.

In [ ]:
from pathlib import Path
from collections import Counter
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import string

from tqdm.auto import tqdm
import logging

logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)

tqdm.pandas()

workdir = Path("..")
df = pd.read_parquet(workdir / "data" / "catalogo-de-teses-e-dissertacoes.parquet")

Teses sem título ou resumo não serão consideradas na análise.

In [ ]:
df = df.dropna(subset=["NM_PRODUCAO", "DS_RESUMO"])

Removemos as seguintes colunas da análise

- `NM_DISCENTE`: Nome do discente
- `DS_URL_TEXTO_COMPLETO`: URL para o texto completo
- `NR_PAGINAS`: Número de páginas
- `NR_VOLUME`: Número do volume	


In [ ]:
df = df.drop(
    columns=["NM_DISCENTE", "DS_URL_TEXTO_COMPLETO", "NR_PAGINAS", "NR_VOLUME"]
)
df = df.drop_duplicates()
df.info()

Verificando a distribuição do tamanho dos textos, podemos observar que 75% dos textos possuem até 2516 caracteres. O maior texto possui 32.767 caracteres. Na próxima seção, vamos trabalhar na identificação de padrões e elementos que podem ser removidos.

In [ ]:
df.loc[:, "DS_RESUMO"] = df["DS_RESUMO"].str.lower()
df = df.sort_values("AN_BASE").drop_duplicates("DS_RESUMO", keep="last")

In [ ]:
df["DS_RESUMO"].str.len().describe().astype(int)

## Limpeza do Resumo

No conjunto de dados,há textos formados apenas por caracteres especiais ou repetidos. Removemos esse padrão de texto, pois não é relevante para a análise.

In [ ]:
df = df.loc[~df["DS_RESUMO"].str.match(r"^(.)\1+$")]  # remove caracteres repetidos

Temos a situação onde o texto é formado majoritariamente por caracteres especiais, como `#`, `*`, `@`, `!`, `?`, `&`, `+`, `=`, `~`, `^`, `%`, `|`, `;`, `:`, `.`. Primeiro calculamos a proporção desses caracteres, mantendo apenas os textos onde a proporção é menor que 0.2.

In [ ]:
df["PROPORCAO_ESPECIAIS"] = (
    df["DS_RESUMO"].str.count(r"[^\w\s]") / df["DS_RESUMO"].str.len()
)
# manter apenas textos com menos de 20% de caracteres especiais
df = df.loc[df["PROPORCAO_ESPECIAIS"] < 0.2]

Outro caso presente no conjunto são termos como `NONONO` ou `AAAAAAAAAA`. Também removemos esses textos.

In [ ]:
df["DS_RESUMO"] = df["DS_RESUMO"].str.replace(r"([\w.*-][\w.*-])\1{2,}", "", regex=True)
df = df.drop_duplicates("DS_RESUMO", keep="last")

Analisando teses com até 50 caracteres, observamos que muitos dos textos não são relevantes para a análise, pois são compostos por caracteres repetidos ou informações com pouco significado. Abaixo, temos alguns exemplos de textos com até 50 caracteres. 

- `este estudo`
- `considerando o momento político...`
- `kklçklçk`
- `kjgutdrs`
- `não contém a informação`
- `fekgh`
- `tgsc`
- `sdfskdjfksj`
- `será inserido após correçoes.`
- `fgfggh`

In [ ]:
for t in df.loc[df["DS_RESUMO"].str.len() <= 50, "DS_RESUMO"].sample(10).values:
    print(f"- `{t}`")

In [ ]:
df = df.loc[df["DS_RESUMO"].str.len() > 50]
df.shape

In [ ]:
words = df["DS_RESUMO"].str.cat(sep=" ").split()
types = Counter(words)

In [ ]:
print(f"Total de palavras: {len(words):,}")
print(f"Tamanho do vocabulário: {len(types):,}")
print(f"Riqueza do corpus: {len(types) / len(words):.2%}")

**Hapax legomena**

Hapax legomena é uma palavra ou expressão que ocorre apenas uma vez dentro de um contexto: seja no registro escrito de uma língua inteira, nas obras de um autor ou em um único texto. Em nosso conjunto de dados, temos 5 milhões de palavras únicas. Como forma de acelerar o processamento e reduzir a complexidade, removemos palavras que ocorrem apenas uma vez no conjunto de dados, pois não são relevantes para a tarefa de extração de tópicos.

In [ ]:
hapaxes = set([word for word, count in types.items() if count == 1])
print(f"Hapax legomena: {len(hapaxes):,}")
print(f"Proporção de hapax legomena: {len(hapaxes) / len(types):.2%}")

In [ ]:
df["DS_RESUMO"] = df["DS_RESUMO"].progress_apply(
    lambda text: " ".join([word for word in text.split() if word not in hapaxes])
)
df = df.drop_duplicates("DS_RESUMO", keep="last")

In [ ]:
df = df.loc[df["DS_RESUMO"].str.len() > 50]
df.shape

Ao analisar alguns resumos, vemos que há textos com informações sobre a entrega da versão final da dissertação ou tese. Abaixo, temos alguns exemplos de textos com informações sobre a entrega da versão final. 

- o aluno ainda disponibilizou a versão final da dissertação.
- o resumo será apresentada na versão final da dissertação.
- o aluno não disponibilizou a dissertação de mestrado.
- tese não enviada à biblioteca ou ainda não catalogada.
- a aluna defendeu, mas ainda não entregou o material final.
- aluna ainda não entregou a versão final da dissertação.
- defesa fechada não autorizada divulgação pela aluna

In [ ]:
df.loc[df["DS_RESUMO"].str.len() < 60, "DS_RESUMO"].sample(10).values

Para lidar com esses textos, baixamos um modelo pré-treinado em português do Word2Vec, o objetivo é identificar sentenças relacionadas à entrega da versão final da dissertação ou tese. Com isso, podemos remover essas sentenças dos textos.

```bash
wget http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s300.zip -O cbow_s300.zip
unzip cbow_s300.zip
```

In [ ]:
def get_sentence_vector(sentence):
    words = sentence.split()
    vectors = [model[word] for word in words if word.isalpha() and word in model]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)


def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

model = KeyedVectors.load_word2vec_format(workdir / "cbow_s300.txt")

In [ ]:
vectors = df["DS_RESUMO"].progress_apply(get_sentence_vector)
vectors = np.stack(vectors)

Abaixo recuperamos as sentenças mais similares à entrega da versão final da dissertação ou tese.


In [ ]:
similarities = cosine_similarity(
    vectors,
    get_sentence_vector("o aluno ainda disponibilizou a versão final da dissertação."),
)
idxs = np.argsort(similarities.ravel())[::-1]
print(similarities[idxs[:50]])
df.iloc[idxs[:50]]["DS_RESUMO"].values
df.loc[(similarities >= 0.0006), "DS_RESUMO"].values

Após observar os textos mais similares, definimos um limiar para identificar sentenças relacionadas à entrega da versão final. 

In [ ]:
texts = [
    "o aluno ainda disponibilizou a versão final da dissertação.",
    "o resumo será apresentada na versão final da dissertação.",
    "o aluno não disponibilizou a dissertação de mestrado.",
    "tese não enviada à biblioteca ou ainda não catalogada.",
    "a aluna defendeu, mas ainda não entregou o material final.",
    "aluna ainda não entregou a versão final da dissertação.",
    "o resumo será enviado juntamente com o texto definitivo da dissertação",
    "a dissertação está passando por correções de conteúdo e será entregue futuramente.",
    "defesa fechada - proibida a divulgação da dissertação por estar em processo de registro de patente",
    "o resumo da dissertação encontra-se no estágio de correções acadêmicas e gráficas, sugeridas pela banca examinadora.",
    "até o momento não foi depositada na secretaria o resumo.",
    "a aluna nao entregou a dissertacao com sua devidas correcoes na secretaria ate esta data.",
    "o referido aluno nao entregou a versao definitiva ainda",
    "aguardando a discente entregar as cópias da versão final da tese com o devido aval de sua orientadora.",
    "a partir da data de defesa, a aluna tem 45 dias para a entrega da versão definitiva da sua dissertação.",
    "até o presente momento não foi entregue o resumo da tese.",
    "encontra-se à disposição no site: ",
    "produção intelectual ainda não foi entregue",
    "não foi entregue pelo discente ao programa a versão final após a defesa",
    "aguardando trabalho final do discente",
]

threshold = 0.00065
for text in texts:
    similarities = cosine_similarity(vectors, get_sentence_vector(text))
    df = df.loc[~(similarities >= threshold)]
    vectors = vectors[~(similarities >= threshold)]

In [ ]:
def preprocess_text(text):
    text = (
        text.lower()
        .translate(str.maketrans("", "", string.punctuation))
        .translate(str.maketrans("", "", string.digits))
        .strip()
    )
    text = " ".join(
        [word for word in text.split() if len(word) > 1 or word in ["a", "o"]]
    )
    return text


df["DS_RESUMO_LIMPO"] = df["DS_RESUMO"].progress_apply(preprocess_text)
df = df.loc[df["DS_RESUMO_LIMPO"].str.len() > 20]
df = df.drop_duplicates("DS_RESUMO_LIMPO", keep="last")

In [ ]:
df.drop(columns=["PROPORCAO_ESPECIAIS"]).to_parquet(
    workdir / "data" / "catalogo-de-teses-e-dissertacoes-limpo.parquet"
)